<a href="https://colab.research.google.com/github/jaimonjacob/mlstarter/blob/main/07_PyTorch_Experiment_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!kaggle datasets download jeffheaton/iris-computer-vision

Dataset URL: https://www.kaggle.com/datasets/jeffheaton/iris-computer-vision
License(s): GNU Lesser General Public License 3.0
 75% 4.00M/5.33M [00:00<00:00, 6.00MB/s]
100% 5.33M/5.33M [00:01<00:00, 5.54MB/s]


In [2]:
!pip install -q split-folders torchinfo

In [3]:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def skip(line, cell):
    return

In [4]:
try:
    from torch.utils.tensorboard import SummaryWriter
except:
    print("[INFO] Couldn't find tensorboard... installing it.")
    !pip install -q tensorboard
    from torch.utils.tensorboard import SummaryWriter

In [5]:
import splitfolders

In [8]:
import torch
from torch import nn
from typing import Dict, List, Tuple
import requests
import zipfile
from pathlib import Path
import matplotlib.pyplot as plt
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import v2
import random
from PIL import Image
from torchinfo import summary
import shutil
import matplotlib.image as mpimg
import os
import splitfolders

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [10]:
# Define the path for the zip file and the destination folder
zip_file_path = 'file.zip'
destination_folder = 'data'

# Create the destination folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

# Extract the contents of the zip file into the destination folder
with zipfile.ZipFile("iris-computer-vision.zip", 'r') as zip_ref:
    zip_ref.extractall(destination_folder)

print(f"Extracted {zip_file_path} into {destination_folder}")

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/iris-computer-vision.zip'

In [ ]:
# !rm -rf data/images
# !rm -rf data/train
# !rm -rf data/test
# !rm -rf data
# !rm -rf data

In [ ]:
input_folder = "data"
output = "/Data" #where you want the split datasets saved. one will be created if it does not exist or none is set

splitfolders.ratio(input_folder, output=output, seed=42, ratio=(.75, .25)) # ratio of split are in order of train/val/test. You can change to whatever you want. For train/val sets only, you could do .75, .25 for example.

In [ ]:
image_path = Path("Data")

train_dir = image_path / "train"
test_dir = image_path / "val"


In [ ]:
import glob
test_image_path_list = list(test_dir.glob("**/*.jpg")) # Change to the correct image extension if needed

# Helper functions

In [ ]:
def visualize_random_images(folder_path, num_images=10):
    # Collect all image paths and their class labels
    image_paths = []
    class_labels = []

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.jpg') or file.endswith('.png'):
                image_paths.append(os.path.join(root, file))
                class_labels.append(os.path.basename(root))

    # Randomly select the specified number of images
    selected_indices = random.sample(range(len(image_paths)), min(num_images, len(image_paths)))
    selected_images = [image_paths[i] for i in selected_indices]
    selected_labels = [class_labels[i] for i in selected_indices]
    print(selected_images)
    # Plot the selected images
    plt.figure(figsize=(20, 10))
    for i in range(len(selected_images)):
        img = mpimg.imread(selected_images[i])
        plt.subplot(2, 5, i + 1)
        plt.imshow(img)
        plt.title(selected_labels[i])
        plt.axis('off')

    plt.show()

In [ ]:
#Function to plot transformed images
def plot_transformed_images(image_paths, transform, n=3, seed=42):
    """Plots a series of random images from image_paths.

    Will open n image paths from image_paths, transform them
    with transform and plot them side by side.

    Args:
        image_paths (list): List of target image paths.
        transform (PyTorch Transforms): Transforms to apply to images.
        n (int, optional): Number of images to plot. Defaults to 3.
        seed (int, optional): Random seed for the random generator. Defaults to 42.
    """
    random.seed(seed)
    random_image_paths = random.sample(image_paths, k=n)
    for image_path in random_image_paths:
        with Image.open(image_path) as f:
            fig, ax = plt.subplots(1, 2)
            ax[0].imshow(f)
            ax[0].set_title(f"Original \nSize: {f.size}")
            ax[0].axis("off")

            # Transform and plot image
            # Note: permute() will change shape of image to suit matplotlib
            # (PyTorch default is [C, H, W] but Matplotlib is [H, W, C])
            transformed_image = transform(f).permute(1, 2, 0)
            ax[1].imshow(transformed_image)
            ax[1].set_title(f"Transformed \nSize: {transformed_image.shape}")
            ax[1].axis("off")

            fig.suptitle(f"Class: {image_path.parent.stem}", fontsize=16)


In [ ]:
def random_visualize_model_results(model, dataset):
    #torch.manual_seed(42)
    fig = plt.figure(figsize=(12,12))
    rows, cols = 4, 4
    for i in range(1, rows*cols+1):
        random_idx = torch.randint(0, len(dataset), size=[1]).item()
        image, label = dataset[random_idx]
        fig.add_subplot(rows, cols, i)
        plt.imshow(image.permute(1, 2, 0))
        model.eval()
        with torch.inference_mode():
            y_logits = model(image.unsqueeze(dim=0).to(device))
            y_predprobs = torch.softmax(y_logits, dim=1)
            y_preds = torch.argmax(y_predprobs, dim=1)
            actual_label = class_names[label]
            pred_label = class_names[y_preds.item()]
        if y_preds.item() == label:
            plt.title(f"Pred: {pred_label} | Actual: {actual_label}", c="g", fontsize=10)
        else:
            plt.title(f"Pred: {pred_label} | Actual: {actual_label}", c="r", fontsize=10)
        plt.axis(False)



In [ ]:
#Functionising the training and test loops
def train_step(model: nn.Module,
               loss_fn: nn.Module,
               dataloader: torch.utils.data.DataLoader,
               optimizer: torch.optim.Optimizer,
               device: torch.device=device):
     #Initiating the train loss as 0 so that we can add to it with each batch
    train_loss = 0
    train_accuracy = 0
    model.to(device)
    #Train loop
    #For each epoch, we will iterate through the batches
    for batch, (X,y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        model.train()
        #1 forward pass
        y_pred = model(X)
        #2 Calculate the loss and accuracy per batch
        loss = loss_fn(y_pred, y)
         # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        accuracy = (y_pred_class == y).sum().item()/len(y_pred)
        # accuracy = accuracy_fn(y_pred.argmax(dim=1), y)
        #Accumulate training loss for every batch
        train_loss += loss
        train_accuracy += accuracy
        #3 Opitmizer zero grad
        optimizer.zero_grad()
        #4 Loss backward
        loss.backward()
        #5 Optimizer step. The model's paramters are updated once per batch, not per epoch
        optimizer.step()

    #Divide total trainig loss/accuracy by length of train data loader - find average
    train_loss /= len(train_dataloader)
    train_accuracy /= len(train_dataloader)
    return train_loss, train_accuracy


def test_step(model: nn.Module,
              loss_fn: nn.Module,
              dataloader: torch.utils.data.DataLoader,
              optimizer: torch.optim.Optimizer,
              device: torch.device=device):
    #Test loop
    test_loss = 0
    test_accuracy = 0
    model.to(device)
    model.eval()
    with torch.inference_mode():
        for batch, (X,y) in enumerate(dataloader):
            X = X.to(device)
            y = y.to(device)
            y_pred = model(X)
            #Find accuracy
            y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
            accuracy = (y_pred_class == y).sum().item()/len(y_pred)
            test_accuracy += accuracy
            #2 Calculate the loss per batch
            test_loss += loss_fn(y_pred, y)
        #Calculate test average loss
        test_accuracy /= len(test_dataloader)
        #Calculate test average accuracy
        test_loss /= len(test_dataloader)
    return test_loss, test_accuracy

In [ ]:
# Creating a sigle train function
from tqdm.auto import tqdm


# model: nn.Module,
# loss_fn: nn.Module,
# accuracy_fn,
# dataloader: torch.utils.data.DataLoader,
# optimizer: torch.optim.Optimizer,
# device: torch.device=device):

# 1. Take in various parameters required for training and test steps
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int = 5,
          device: device,
          writer: torch.utils.tensorboard.writer.SummaryWriter,
          ):


    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }

    train_start_on_gpu = timer()
    for epoch in tqdm(range(epochs)):
        train_loss, train_accuracy = train_step(model, loss_fn, train_dataloader, optimizer, device)
        test_loss, test_accuracy = test_step(model, loss_fn, test_dataloader, optimizer, device)

        print(
                f"Epoch: {epoch} | "
                f"train_loss: {train_loss:.4f} | "
                f"train_acc: {train_accuracy:.4f} | "
                f"test_loss: {test_loss:.4f} | "
                f"test_acc: {test_accuracy:.4f}"
            )
        results["train_loss"].append(train_loss.item())
        results["train_acc"].append(train_accuracy)
        results["test_loss"].append(test_loss.item())
        results["test_acc"].append(test_accuracy)

        ### New: Experiment tracking ###
        # Add loss results to SummaryWriter
        writer.add_scalars(main_tag="Loss",
                           tag_scalar_dict={"train_loss": train_loss,
                                            "test_loss": test_loss},
                           global_step=epoch)

        # Add accuracy results to SummaryWriter
        writer.add_scalars(main_tag="Accuracy",
                           tag_scalar_dict={"train_acc": train_accuracy,
                                            "test_acc": test_accuracy},
                           global_step=epoch)

        # Track the PyTorch model architecture
        writer.add_graph(model=model,
                         # Pass in an example input
                         input_to_model=torch.randn(32, 3, 224, 224).to(device))

    # Close the writer
    writer.close()
    ### End new ###


    train_end_on_gpu = timer()
    total_train_time_model_0 = print_train_time(start=train_start_on_gpu, end=train_end_on_gpu)
    return results


In [ ]:
#Creating a function to plot loss
def plot_loss_curves(results: Dict[str, List[float]]):
    """Plots training curves of a results dictionary.

    Args:
        results (dict): dictionary containing list of values, e.g.
            {"train_loss": [...],
             "train_acc": [...],
             "test_loss": [...],
             "test_acc": [...]}
    """

    # Get the loss values of the results dictionary (training and test)
    loss = results['train_loss']
    test_loss = results['test_loss']

    # Get the accuracy values of the results dictionary (training and test)
    accuracy = results['train_acc']
    test_accuracy = results['test_acc']

    # Figure out how many epochs there were
    epochs = range(len(results['train_loss']))

    # Setup a plot
    plt.figure(figsize=(15, 7))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label='train_loss')
    plt.plot(epochs, test_loss, label='test_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label='train_accuracy')
    plt.plot(epochs, test_accuracy, label='test_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend();

In [ ]:
from timeit import default_timer as timer

def print_train_time(start: float,
                    end: float,
                    device: torch.device=None):
    """
    Prints the different between start and end of training

    Args:
    Start, end, device

    Returns:
    Time in string
    """
    total_time = end - start
    print (f"The time spent on training on {device}: {total_time: .3f} seconds")
    return total_time


In [ ]:
# Set seeds
def set_seeds(seed: int=42):
    """Sets random sets for torch operations.

    Args:
        seed (int, optional): Random seed to set. Defaults to 42.
    """
    # Set the seed for general torch operations
    torch.manual_seed(seed)
    # Set the seed for CUDA torch operations (ones that happen on the GPU)
    torch.cuda.manual_seed(seed)

In [ ]:
def create_writer(experiment_name: str,
                  model_name: str,
                  extra: str=None) -> torch.utils.tensorboard.writer.SummaryWriter():
    """Creates a torch.utils.tensorboard.writer.SummaryWriter() instance saving to a specific log_dir.

    log_dir is a combination of runs/timestamp/experiment_name/model_name/extra.

    Where timestamp is the current date in YYYY-MM-DD format.

    Args:
        experiment_name (str): Name of experiment.
        model_name (str): Name of model.
        extra (str, optional): Anything extra to add to the directory. Defaults to None.

    Returns:
        torch.utils.tensorboard.writer.SummaryWriter(): Instance of a writer saving to log_dir.

    Example usage:
        # Create a writer saving to "runs/2022-06-04/data_10_percent/effnetb2/5_epochs/"
        writer = create_writer(experiment_name="data_10_percent",
                               model_name="effnetb2",
                               extra="5_epochs")
        # The above is the same as:
        writer = SummaryWriter(log_dir="runs/2022-06-04/data_10_percent/effnetb2/5_epochs/")
    """
    from datetime import datetime
    import os

    # Get timestamp of current date (all experiments on certain day live in same folder)
    timestamp = datetime.now().strftime("%Y-%m-%d") # returns current date in YYYY-MM-DD format

    if extra:
        # Create log directory path
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name, extra)
    else:
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name)

    print(f"[INFO] Created SummaryWriter, saving to: {log_dir}...")
    return SummaryWriter(log_dir=log_dir)

# Visualize

In [ ]:
visualize_random_images(test_dir)

In [ ]:
visualize_random_images(train_dir)

# USing Efficientnet

In [ ]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT # .DEFAULT = best available weights from pretraining on ImageNet
auto_transforms = weights.transforms()
auto_transforms

In [ ]:
manual_transforms = transforms.Compose([
    transforms.Resize((224, 224)), # 1. Reshape all images to 224x224 (though some models may require different sizes)
    transforms.ToTensor(), # 2. Turn image values to between 0 & 1
])

In [ ]:
# 1. Load and transform data
train_data = datasets.ImageFolder(root=train_dir, transform=auto_transforms )
test_data = datasets.ImageFolder(root=test_dir, transform=auto_transforms )
class_names = train_data.classes

# Setup batch size and number of workers
BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()
print(f"Creating DataLoader's with batch size {BATCH_SIZE} and {NUM_WORKERS} workers.")

# Create DataLoader's
train_dataloader = DataLoader(train_data,
                                     batch_size=BATCH_SIZE,
                                     shuffle=True,
                                     num_workers=NUM_WORKERS,
                              pin_memory=True,)

test_dataloader = DataLoader(test_data,
                                    batch_size=BATCH_SIZE,
                                    shuffle=False,
                                    num_workers=NUM_WORKERS,
                             pin_memory=True)

train_dataloader, test_dataloader, class_names

In [ ]:
# image_path_list = list(image_path.glob("*/*/*.jpg"))
plot_transformed_images(test_image_path_list,
                        transform=auto_transforms,
                        n=3)

In [ ]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT # .DEFAULT = best available weights
model = torchvision.models.efficientnet_b0(weights=weights).to(device)

In [ ]:
summary(model=model,
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

In [ ]:
# Freeze all base layers in the "features" section of the model (the feature extractor) by setting requires_grad=False
for param in model.features.parameters():
    param.requires_grad = False


In [11]:
# Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Get the length of class_names (one output unit for each class)
output_shape = len(class_names)

# Recreate the classifier layer and seed it to the target device
model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(in_features=1280,
                    out_features=output_shape, # same number of output units as our number of classes
                    bias=True)).to(device)

NameError: name 'class_names' is not defined

In [ ]:
# # Do a summary *after* freezing the features and changing the output classifier layer (uncomment for actual output)
summary(model,
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape" (batch_size, color_channels, height, width)
        verbose=0,
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

In [ ]:
random_visualize_model_results(model, test_data)

In [ ]:
loss_fn_0 = nn.CrossEntropyLoss()
optimizer_0 = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Create an example writer
example_writer = create_writer(experiment_name="data_iris-computer-vision",
                               model_name="effnetb0",
                               extra="10_epochs")

In [ ]:
set_seeds()
model_results = train(model = model,
          train_dataloader = train_dataloader,
          test_dataloader = test_dataloader,
          optimizer = optimizer_0,
          loss_fn = loss_fn_0,
          epochs = 10,
          writer = example_writer)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs